[Link to the destination](#the_destination)

## NOW FOCUSED ON OWN-OCCUPIED SINGLE FAMILY

### updates since last version:

Added new features pertaining to aggregate stats. Best rfc for 2010-15 had nearly 50/50 accuracy

Now focus on mappping / visualization and timing effects

In [1]:
import pandas as pd
import numpy as np
import shapely
import fiona
from shapely import geometry
from shapely.geometry import Polygon
from shapely.geometry import MultiPolygon
from shapely.geometry import Point
import json
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline
import mplleaflet as mpl
import pysal as ps


In [2]:
# IMPORT MODULES FOR REGRESSION / CLASSIFICATION


from sklearn.preprocessing import PolynomialFeatures, StandardScaler, Imputer
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.cross_validation import cross_val_score, KFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score
from sklearn.grid_search import GridSearchCV

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.svm import SVC



/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### Start with most recent backup copy of the combined dataframe

In [3]:
model_file = pd.read_csv('/Users/DavidStier/Downloads/Model_File_CT-backup4.csv')

/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (171,172,212,224,231,256,306,309) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
model_file.shape

(92052, 365)

### Clean up 'legacy issues' from imported csv

In [5]:
first_pass_model_file_col_list = model_file.columns.tolist()
first_pass_model_file_dtypes_list = model_file.dtypes.tolist()
first_pass_model_file_first_Row = model_file[1:2]
first_pass_model_file_first_Row.to_csv('/Users/DavidStier/Downloads/first_pass_model_filerow7.csv')

In [6]:
## 
# Model file had included 2 features that were proxies for having installed solar
##

model_file.drop(['Valuation_all_sol_prmts'], inplace = True, axis = 1)
model_file.drop(['NUM_PERMITS_for_solar'], inplace = True, axis = 1)

## 
#  Model file 'neighborhood' and Outbuilding 1-3 columns had mixed content types - specifically the # zero was entered for some rows
##

model_file['neighborhood'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_1'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_2'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_3'].replace(0,'Not Provided',inplace=True)
model_file['Outbldg_3'].replace('0','Not Provided',inplace=True)

##
# Some census tract columns had 'nan' instead of NaN and Dtype needs to be Float
##

model_file['ctbdp040var_HC01_VC66'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC66'] = model_file['ctbdp040var_HC01_VC66'].astype(float)

model_file['ctbdp040var_HC01_VC67'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC67'] = model_file['ctbdp040var_HC01_VC67'].astype(float)

model_file['ctbdp040var_HC01_VC143'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC143'].replace(['4,000+'],4000, inplace = True) 
model_file['ctbdp040var_HC01_VC143'] = model_file['ctbdp040var_HC01_VC143'].astype(float)

model_file['ctbdp040var_HC01_VC151'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC151'].replace(['1,000+'],1000, inplace = True) 
model_file['ctbdp040var_HC01_VC151'].replace(['100-'],100, inplace = True) 
model_file['ctbdp040var_HC01_VC151'] = model_file['ctbdp040var_HC01_VC151'].astype(float)

model_file['ctbdp040var_HC01_VC185'].replace(['-'],0, inplace = True) 
model_file['ctbdp040var_HC01_VC185'].replace(['100-'],100, inplace = True) 
model_file['ctbdp040var_HC01_VC185'].replace(['2,000+'],2000, inplace = True) 
model_file['ctbdp040var_HC01_VC185'] = model_file['ctbdp040var_HC01_VC185'].astype(float)

model_file['ctb19049var_HD01_VD03'].replace(['2,500-'],2500, inplace = True) 
model_file['ctb19049var_HD01_VD03'].replace(['-'],0, inplace = True) 
model_file['ctb19049var_HD01_VD03'] = model_file['ctb19049var_HD01_VD03'].astype(float)

model_file['ctb19049var_HD01_VD06'].replace(['-'],0, inplace = True) 
model_file['ctb19049var_HD01_VD06'] = model_file['ctb19049var_HD01_VD06'].astype(float)

model_file['ctbdp040var_HC01_VC125'].replace(['1,000,000+'],1000000, inplace=True)  #nan and '-' and 1,000,000+
model_file['ctbdp040var_HC01_VC125'].replace(['-'],0, inplace = True)  #nan and '-' and 1,000,000+
model_file['ctbdp040var_HC01_VC125'] = model_file['ctbdp040var_HC01_VC125'].astype(float)



In [7]:
##
# Remove other permit activity if parcel installed solar that same year
# some homes may have had other electrical or structural work done at the same time as when they installed solar


reset_permit_info  =   {2016 : ['NOT_Top2016PermTypdollars','NOT_Top2016PermTypNum','Top2016PermTypdollars', 'Top2016PermTypNum'],
                        2015  : ['NOT_Top2015PermTypdollars','NOT_Top2015PermTypNum','Top2015PermTypdollars', 'Top2015PermTypNum'],
                        2014  : ['NOT_Top2014PermTypdollars','NOT_Top2014PermTypNum','Top2014PermTypdollars', 'Top2014PermTypNum'],
                        2013  : ['NOT_Top2013PermTypdollars','NOT_Top2013PermTypNum','Top2013PermTypdollars', 'Top2013PermTypNum'],
                        2012  : ['NOT_Top2012PermTypdollars','NOT_Top2012PermTypNum','Top2012PermTypdollars', 'Top2012PermTypNum'],
                        2011  : ['NOT_Top2011PermTypdollars','NOT_Top2011PermTypNum','Top2011PermTypdollars', 'Top2011PermTypNum'], 
                        2010  : ['NOT_Top2010PermTypdollars','NOT_Top2010PermTypNum','Top2010PermTypdollars', 'Top2010PermTypNum']}

for key, value in reset_permit_info.items():
    model_file.loc[model_file['Year_SOLAR'] == key,  value] = 0


In [8]:
##
# Check to make sure that the calc's affected the #s appropriately (eyeball check)
#

print model_file['NOT_Top2015PermTypdollars'].sum()
print model_file['NOT_Top2014PermTypdollars'].sum()
print '>>>>>>>>>>'
print model_file['Top2015PermTypdollars'].sum()
print model_file['Top2014PermTypdollars'].sum()


5451990761.01
3846780651.87
>>>>>>>>>>
1134857483.1
1052600682.56


## Add 1/4 mile square grids

In [9]:
min_lat = 42.22
max_lat = 42.4
min_lon = -71.2
max_lon = -70.97
Qlat_incr = 0.001812
Qlon_incr = 0.002438

#Now that lat and lon are associated with the PIDs we want, use lambda on model_file to call on the PID_LAT_LON dframe
model_file['Qmile_grid_cell_R']  = (model_file['Longitude']- min_lon) // Qlon_incr
model_file['Qmile_grid_cell_C'] = (model_file['Latitude']- min_lat) // Qlat_incr
model_file['Qmile_grid_cell'] =  zip(model_file['Qmile_grid_cell_C'],model_file['Qmile_grid_cell_R'])


In [10]:
model_file.shape

(92052, 366)

In [11]:
len(model_file['hmile_grid_cell'].unique())

804

In [12]:
model_file['Year_SOLAR'].unique()

array([   nan,  2015.,  2011.,  2014.,  2016.,  2012.,  2013.,  2010.])

In [13]:
# save out a copy of the model datafile:
# model_file.to_csv('/Users/DavidStier/Downloads/Model_File-backup_TUESDAY.csv')
#model_file.to_json('/Users/DavidStier/Downloads/Model_Filebackup2.json')
# describe_file = model_file.describe()
# describe_file.to_csv('/Users/DavidStier/Downloads/Ctractcombined_described2_TUESDAY.csv')

## Bring in census tract and other shapefiles for leaflet

## Feature engineering - parcel level data

In [14]:
'''
# Possible new features to add based on all properties in addr_group:

model_file['num_block_SFR_remodeled_pre_1980']
model_file['num_block_SFR_remodeled_1980_89']
model_file['num_block_SFR_remodeled_1990_99']
model_file['num_block_SFR_remodeled_2000_05']
model_file['num_block_SFR_remodeled_2006_10']
model_file['num_block_SFR_remodeled_2011_15']

ASSESSED VALUE percentiles
model_file['bldg_vs_land_value_ratio']
'''


"\n# Possible new features to add based on all properties in addr_group:\n\nmodel_file['num_block_SFR_remodeled_pre_1980']\nmodel_file['num_block_SFR_remodeled_1980_89']\nmodel_file['num_block_SFR_remodeled_1990_99']\nmodel_file['num_block_SFR_remodeled_2000_05']\nmodel_file['num_block_SFR_remodeled_2006_10']\nmodel_file['num_block_SFR_remodeled_2011_15']\n\nASSESSED VALUE percentiles\nmodel_file['bldg_vs_land_value_ratio']\n"

In [15]:
#model_file['tot_rooms_per_k_lot_sf'] = [model_file['R_TOTAL_RM'] / model_file['LAND_SF'] if x > 0 else 0 for x in model_file['LAND_SF']]
# model_file['yrs_since_2016_when_remodeled2'] = 0.0
# for i in range(92052):
#     if model_file['YR_REMOD'][i] > 0:
#         model_file['yrs_since_2016_when_remodeled2'][i] = 2016 - model_file['YR_REMOD'][i]
#     else:
#         model_file['yrs_since_2016_when_remodeled2'][i] = 0.0
# model_file['yrs_since_2016_when_remodeled2'].value_counts()

In [16]:
#model_file['bldg_vs_land_value_ratio'] = model_file['AV_BLDG'] / model_file['AV_LAND'] - did not use as it can go to infinity
model_file['home_age_in_2016'] = 2016 - model_file['YR_BUILT']

                                                
                                                

model_file['home_age_when_remodeled'] = model_file['YR_REMOD'] - model_file['YR_BUILT']
model_file['yrs_since_2016_when_remodeled'] = 2016 - model_file['YR_REMOD']

model_file['bathrooms_all_qty'] = model_file['R_FULL_BTH'] + model_file['R_HALF_BTH']
model_file['shape_vs_length'] = model_file['SHAPESTAre'] / model_file['SHAPESTLen']
model_file['tot_rooms_per_k_bldg_sf'] = model_file['R_TOTAL_RM'] / model_file['LIVING_ARE'] 
model_file['is_solar'] = [1 if x > 0 else 0 for x in model_file['Year_SOLAR'] ]

# Convert Owner Occupied to 0/1 from y/n so that it can be aggregated:
model_file['OWN_OCC'] = [1 if x == 'Y' else 0 for x in model_file['OWN_OCC'] ]

# Dummify certain categorical columns now:



#model_file['bldg_vs_land_size_ratio'] = model_file['LIVING_ARE'] / model_file['LAND_SF'] - this goes to infinity
#model_file['home_age_last_top_permitType_done'] = 
model_file.shape

(92052, 373)

In [17]:
Heat_Type_dummied = pd.get_dummies(model_file['Heat_Type'],prefix = 'Heat_Type_')
LU_dummied =pd.get_dummies(model_file['LU'],prefix = 'LU_')
R_ROOF_TYP_dummied = pd.get_dummies(model_file['R_ROOF_TYP'],prefix = 'R_ROOF_TYPE_')

type(Heat_Type_dummied)

pandas.core.frame.DataFrame

In [18]:
model_file = pd.concat([model_file, Heat_Type_dummied, LU_dummied, R_ROOF_TYP_dummied], axis = 1)

In [19]:
model_file.shape

(92052, 407)

### Select 'top' features to use for merging the aggregated calcs.  This will limit the # of new features
Based on early RandomForest Classifier results, have selected those features that had at least .002 contribution

In [20]:
top_aggr_features = ['addr_group','hmile_grid_cell','Qmile_grid_cell','neighborhood','FIPS','NOT_Top2015PermTypNum','NOT_Top2015PermTypdollars','NOT_Top2014PermTypdollars',
                     'NOT_Top2013PermTypdollars','NOT_Top2014PermTypNum','GROSS_AREA','NOT_Top2013PermTypNum',
                     'SHAPESTLen','NOT_Top2012PermTypdollars','AV_LAND','home_age_when_remodeled','LIVING_ARE',
                     'home_age_in_2016','bldg_vs_land_value_ratio','tot_rooms_per_k_bldg_sf','LAND_SF','YR_REMOD',
                     'NOT_Top2012PermTypNum','YR_BUILT','shape_vs_length','NUM_FLOORS', 'OWN_OCC','qmile_grid_cell_R',
                     'yrs_since_2016_when_remodeled','Outbldg_1_Sqft','Exterior_Wall_Alum/Vinyl',
                     'NOT_Top2011PermTypdollars','R_TOTAL_RM','AV_BLDG','ctb1001var_HD01_VD41','ctb25093var_HD01_VD21',
                     'R_BDRMS','SHAPESTAre','st_num_group','Thr12014to2015_Street-Sidewalk','qmile_grid_cell_C',
                     'NOT_Top2010PermTypdollars','Thr12014to2015_excluded','R_FULL_BTH','PID','Heat_Type__Electric',
                     'LU__Residential Condo Unit','LU__Mixed Use Res/Comm', 'LU__Commercial','LU__Apartments 7 Units above','LU__Apartments 4-6 Units', 
                     'LU__Residential Single Family', 'LU__Residential Three Family', 'LU__Residential Two Family',
                     'LU__Residential Land', 'LU__Exempt','LU__Commercial Land', 'LU__Commercial Condo Unit','LU__Industrial',
                     'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Trees','Thr12014to2015_Trees','Thr12011to2013_Housing','Thr12011to2013_Building','Thr12014to2015_Building','Thr12011to2013_excluded', 'Thr12014to2015_excluded','Thr12015_excluded',
                     'Thr12014to2015_Housing','Thr12014to2015_Recycling-Pickup', 'is_solar','Thr12011_Recycling-Pickup','Thr12012_Recycling-Pickup','Thr12013_Recycling-Pickup','Thr12014_Recycling-Pickup','Thr12015_Recycling-Pickup', 'Thr12012_Trees','Thr12013_Trees' ,'Thr12014_Trees']

In [21]:
model_file_aggr_cols = pd.DataFrame()
for i in model_file.columns:
    if i in top_aggr_features:
        model_file_aggr_cols[i] = model_file[i]

In [22]:
model_file_aggr_cols.head(4).T

,0,1,2,3
AV_BLDG,235300,281200,244700,227800
AV_LAND,129900,121000,121400,121900
GROSS_AREA,3353,3299,3392,3108
LAND_SF,1150,1150,1150,1150
LIVING_ARE,2202,2307,2268,2028
NUM_FLOORS,3,3,3,3
OWN_OCC,1,0,0,0
PID,100001000,100002000,100003000,100004000
R_BDRMS,6,3,5,5
R_FULL_BTH,3,3,3,3


In [23]:

model_file_addr_count = model_file_aggr_cols.groupby('addr_group',as_index = False).count()
#model_file_addr_num_recs = model_file.groupby('addr_group',as_index = False).size() # this is just the same value across all col's
model_file_addr_mean = model_file_aggr_cols.groupby('addr_group',as_index = False).mean()
model_file_addr_median = model_file_aggr_cols.groupby('addr_group',as_index = False).median()
#model_file_addr_mode = model_aggr_on.groupby('addr_group').mode()
#model_file_addr_sdev = model_aggr_on.groupby('addr_group',as_index = False).std()
model_file_addr_sum = model_file_aggr_cols.groupby('addr_group',as_index = False).sum()

model_file_halfMile_count = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).count()
model_file_halfMile_mean = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).mean()
model_file_halfMile_median = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).median()
model_file_halfMile_sum = model_file_aggr_cols.groupby('hmile_grid_cell',as_index = False).sum()

model_file_QtrMile_count = model_file_aggr_cols.groupby('Qmile_grid_cell',as_index = False).count()
model_file_QtrMile_mean = model_file_aggr_cols.groupby('Qmile_grid_cell',as_index = False).mean()
model_file_QtrMile_median = model_file_aggr_cols.groupby('Qmile_grid_cell',as_index = False).median()
model_file_QtrMile_sum = model_file_aggr_cols.groupby('Qmile_grid_cell',as_index = False).sum()

model_file_nbrhood_count = model_file_aggr_cols.groupby('neighborhood',as_index = False).count()
model_file_nbrhood_mean = model_file_aggr_cols.groupby('neighborhood',as_index = False).mean()
model_file_nbrhood_median = model_file_aggr_cols.groupby('neighborhood',as_index = False).median()
#model_file_nbrhood_mode = model_aggr_on.groupby('neighborhood',as_index = False).mode()
#model_file_nbrhood_sdev = model_aggr_on.groupby('neighborhood',as_index = False).std()
model_file_nbrhood_sum = model_file_aggr_cols.groupby('neighborhood',as_index = False).sum()

model_file_centrct_count = model_file_aggr_cols.groupby('FIPS',as_index = False).count()
model_file_centrct_mean = model_file_aggr_cols.groupby('FIPS',as_index = False).mean()
model_file_centrct_median = model_file_aggr_cols.groupby('FIPS',as_index = False).median()
#model_file_centrct_mode = model_aggr_on.groupby('FIPS',as_index = False).mode()
#model_file_centrct_sdev = model_aggr_on.groupby('FIPS',as_index = False).std()
model_file_centrct_sum = model_file_aggr_cols.groupby('FIPS',as_index = False).sum()


"""
###

The sdev won't work if there are any string values in the df
##
"""

"\n###\n\nThe sdev won't work if there are any string values in the df\n##\n"

In [24]:
model_file.shape

(92052, 407)

### Create joined file aggregate level stats (eg 1 table for sum, count, median @ level = ____ and merge that to the main model file

In [25]:
# first, rename columns so that merge will not try to add suffixes (or not find) a column:
# Note, the 2 lists are for reference. I just cut and pasted into the lambda functions below:

# aggregate_tables = ['model_file_addr_count','model_file_addr_mean','model_file_addr_median',
#                     'model_file_halfMile_count','model_file_halfMile_mean','model_file_halfMile_median',
#                     'model_file_nbrhood_count','model_file_nbrhood_mean','model_file_nbrhood_median',
#                    'model_file_centrct_count', 'model_file_centrct_mean', 'model_file_centrct_median']

# aggregate_suffixes = ['addr_ct','addr_av','add_md','hmil_ct','hmil_av','hmil_md',
#                     'nbhd_ct','nbhd_av','nbhd_md','ctrt_ct', 'ctrt_av', 'ctrt_md']

# The lambda functions:
model_file_addr_count.rename(columns = lambda x: x + '_FOR_addr_cnt', inplace = True)
model_file_addr_mean.rename(columns = lambda x: x + '_FOR_addr_avg', inplace = True)
model_file_addr_median.rename(columns = lambda x: x + '_FOR_addr_med', inplace = True)
model_file_QtrMile_count.rename(columns = lambda x: x + '_FOR_Qmil_cnt', inplace = True)
model_file_QtrMile_mean.rename(columns = lambda x: x + '_FOR_Qmil_avg', inplace = True)
model_file_QtrMile_median.rename(columns = lambda x: x + '_FOR_Qmil_med', inplace = True)
model_file_halfMile_count.rename(columns = lambda x: x + '_FOR_hmil_cnt', inplace = True)
model_file_halfMile_mean.rename(columns = lambda x: x + '_FOR_hmil_avg', inplace = True)
model_file_halfMile_median.rename(columns = lambda x: x + '_FOR_hmil_med', inplace = True)
model_file_nbrhood_count.rename(columns = lambda x: x + '_FOR_nbhd_cnt', inplace = True)
model_file_nbrhood_mean.rename(columns = lambda x: x + '_FOR_nbhd_avg', inplace = True)
model_file_nbrhood_median.rename(columns = lambda x: x + '_FOR_nbhd_med', inplace = True)
model_file_centrct_count.rename(columns = lambda x: x + '_FOR_ctrt_cnt', inplace = True)
model_file_centrct_mean.rename(columns = lambda x: x + '_FOR_ctrt_avg', inplace = True)
model_file_centrct_median.rename(columns = lambda x: x + '_FOR_ctrt_med', inplace = True)


In [26]:

# second, join one and two files as :
addr_group_step2 = pd.merge(model_file_addr_count, model_file_addr_median, left_on = 'addr_group_FOR_addr_cnt', right_on = 'addr_group_FOR_addr_med', how = 'left', suffixes=('', '_y'))
# # # add in third for :
addr_group_final = pd.merge(addr_group_step2, model_file_addr_mean, left_on = 'addr_group_FOR_addr_cnt', right_on ='addr_group_FOR_addr_avg', how = 'left' )
# # # merge the final to model_file
model_file_agg_ip = pd.merge(model_file, addr_group_final, left_on = 'addr_group', right_on ='addr_group_FOR_addr_cnt', how = 'left')

#repeat procedure for grids and merge to main file
grid_step2 = pd.merge(model_file_QtrMile_count, model_file_QtrMile_median,  left_on = 'Qmile_grid_cell_FOR_Qmil_cnt', right_on = 'Qmile_grid_cell_FOR_Qmil_med', how = 'left')
grid_final = pd.merge(grid_step2, model_file_QtrMile_mean,  left_on = 'Qmile_grid_cell_FOR_Qmil_cnt', right_on = 'Qmile_grid_cell_FOR_Qmil_avg', how = 'left')
model_file_agg_ip2 = pd.merge(model_file_agg_ip, grid_final, left_on = 'Qmile_grid_cell', right_on = 'Qmile_grid_cell_FOR_Qmil_cnt', how = 'left')

nhood_step2 = pd.merge(model_file_nbrhood_count, model_file_nbrhood_median, left_on = 'neighborhood_FOR_nbhd_cnt', right_on = 'neighborhood_FOR_nbhd_med', how = 'left')
nhood_final = pd.merge(nhood_step2, model_file_nbrhood_mean , left_on = 'neighborhood_FOR_nbhd_cnt', right_on = 'neighborhood_FOR_nbhd_avg', how = 'left')
model_file_agg_ip3 = pd.merge(model_file_agg_ip2, nhood_final, left_on = 'neighborhood', right_on = 'neighborhood_FOR_nbhd_cnt', how = 'left')

ctrt_step2 = pd.merge(model_file_centrct_count, model_file_centrct_median, left_on = 'FIPS_FOR_ctrt_cnt', right_on = 'FIPS_FOR_ctrt_med', how = 'left')
ctrt_final = pd.merge(ctrt_step2, model_file_centrct_mean, left_on = 'FIPS_FOR_ctrt_cnt', right_on = 'FIPS_FOR_ctrt_avg', how = 'left')
model_agg_ip4 = pd.merge(model_file_agg_ip3, ctrt_final, left_on = 'FIPS', right_on = 'FIPS_FOR_ctrt_cnt', how = 'left')

# added hmile grid 'back in' to the mix, put at end of merge process to save time retyping code

Hgrid_step2 = pd.merge(model_file_halfMile_count, model_file_halfMile_median,  left_on = 'hmile_grid_cell_FOR_hmil_cnt', right_on = 'hmile_grid_cell_FOR_hmil_med', how = 'left')
Hgrid_final = pd.merge(Hgrid_step2, model_file_halfMile_mean,  left_on = 'hmile_grid_cell_FOR_hmil_cnt', right_on = 'hmile_grid_cell_FOR_hmil_avg', how = 'left')
model_file_agg_ip5 = pd.merge(model_agg_ip4, Hgrid_final, left_on = 'hmile_grid_cell', right_on = 'hmile_grid_cell_FOR_hmil_cnt', how = 'left')


### delete 'in-process' files and rename final file back to model_file

In [27]:
del addr_group_step2
del addr_group_final
del model_file_agg_ip
del grid_step2
del grid_final
del model_file_agg_ip2
del nhood_step2
del nhood_final
del model_file_agg_ip3
del ctrt_step2
del ctrt_final
del model_agg_ip4
del Hgrid_step2
del Hgrid_final



model_file = model_file_agg_ip5

In [28]:
model_file.shape

(92052, 1485)

In [29]:
del model_file_agg_ip5

## confirm all joined columns are ready for get_dummies and remove non-informative col's

In [30]:
aggregated_model_file_columns_list = model_file.columns.tolist()
aggregated_model_file_dtypes_list = model_file.dtypes.tolist()
aggregated_model_file_first_Row = model_file[1:2]
aggregated_model_file_first_Row.to_csv('/Users/DavidStier/Downloads/aggregated_model_filerow1QMILE7.csv')
aggregated_model_file_dtypes_list

[dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('bool'),
 dtype('float64'),
 dtype('O'),
 dtype('O'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 d

In [31]:
model_value_cols = model_file.select_dtypes(include=['float64', 'int64','int','float']).columns.tolist()

In [32]:
#model_value_cols

### drop aggregate 'name' columns 

In [33]:
aggr_name_cols_to_drop =[ 
'addr_group_FOR_addr_cnt', 'Qmile_grid_cell_FOR_addr_cnt', 'hmile_grid_cell_FOR_addr_cnt', 'neighborhood_FOR_addr_cnt', 'FIPS_FOR_addr_cnt',
'addr_group_FOR_addr_med', 'FIPS_FOR_addr_med',
'addr_group_FOR_addr_avg', 'FIPS_FOR_addr_avg',

'Qmile_grid_cell_FOR_Qmil_cnt', 'hmile_grid_cell_FOR_Qmil_cnt', 'neighborhood_FOR_Qmil_cnt', 'FIPS_FOR_Qmil_cnt',
'Qmile_grid_cell_FOR_Qmil_med','FIPS_FOR_Qmil_med',
'Qmile_grid_cell_FOR_Qmil_avg','FIPS_FOR_Qmil_avg',

'hmile_grid_cell_FOR_hmil_cnt', 'Qmile_grid_cell_FOR_hmil_cnt', 'neighborhood_FOR_hmil_cnt', 'FIPS_FOR_hmil_cnt',
'hmile_grid_cell_FOR_hmil_med','FIPS_FOR_hmil_med',
'hmile_grid_cell_FOR_hmil_avg','FIPS_FOR_hmil_avg',

'neighborhood_FOR_nbhd_cnt', 'Qmile_grid_cell_FOR_nbhd_cnt', 'hmile_grid_cell_FOR_nbhd_cnt', 'FIPS_FOR_nbhd_cnt',
'neighborhood_FOR_nbhd_med','FIPS_FOR_nbhd_med',
'neighborhood_FOR_nbhd_avg','FIPS_FOR_nbhd_avg',

'FIPS_FOR_ctrt_cnt', 'Qmile_grid_cell_FOR_ctrt_cnt', 'hmile_grid_cell_FOR_ctrt_cnt', 'neighborhood_FOR_ctrt_cnt', 
'FIPS_FOR_ctrt_med', 
'FIPS_FOR_ctrt_avg', 
    
'PID_FOR_addr_avg', 'PID_FOR_addr_med',
'PID_FOR_Qmil_avg', 'PID_FOR_Qmil_med',
'PID_FOR_hmil_avg', 'PID_FOR_hmil_med',
'PID_FOR_nbhd_avg', 'PID_FOR_nbhd_med',
'PID_FOR_ctrt_avg', 'PID_FOR_ctrt_med',
'GEO.id2'
]

model_file.drop(aggr_name_cols_to_drop, inplace = True, axis = 1)

##   Add features based on aggregated stats

In [34]:
testing_pivot = pd.pivot_table(model_file, index = 'addr_group', columns = 'LU', aggfunc = 'size')
testing_pivot.query('addr_group == ["A ST-2.0"]')#['Commercial Land']
# 'Manager == ["Debra Henley"]')

LU,Agricultural/Horticultural,Apartments 4-6 Units,Apartments 7 Units above,Commercial,Commercial Condo Unit,Commercial Land,Condominium Parking,Exempt,Exempt 121A,Industrial,Mixed Use Res/Comm,Residential Condo Unit,Residential Land,Residential Single Family,Residential Three Family,Residential Two Family,XX
addr_group,,,,,,,,,,,,,,,,,
A ST-2.0,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
addr_LU_pivot = pd.DataFrame(testing_pivot)
addr_LU_pivot.head()

LU,Agricultural/Horticultural,Apartments 4-6 Units,Apartments 7 Units above,Commercial,Commercial Condo Unit,Commercial Land,Condominium Parking,Exempt,Exempt 121A,Industrial,Mixed Use Res/Comm,Residential Condo Unit,Residential Land,Residential Single Family,Residential Three Family,Residential Two Family,XX
addr_group,,,,,,,,,,,,,,,,,
A ST-0.0,NaN,NaN,NaN,4.0,NaN,4.0,NaN,1.0,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
A ST-1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A ST-2.0,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A ST-3.0,NaN,NaN,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
A ST-nan,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
model_file.shape

(92052, 1435)

In [37]:
#addr_LU_pivot.to_csv('/Users/DavidStier/Downloads/LUpivot.csv')
addrLUfrom_CSV = pd.read_csv('/Users/DavidStier/Downloads/LUpivot.csv')
addrLUfrom_CSV.head()

,addr_group,Agricultural/Horticultural,Apartments 4-6 Units,Apartments 7 Units above,Commercial,Commercial Condo Unit,Commercial Land,Condominium Parking,Exempt,Exempt 121A,Industrial,Mixed Use Res/Comm,Residential Land,Residential Single Family,Residential Three Family,Residential Two Family,XX
0,A ST-0.0,NaN,NaN,NaN,4.0,NaN,4.0,NaN,1.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN
1,A ST-1.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A ST-2.0,NaN,NaN,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A ST-3.0,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
4,A ST-nan,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
addrLUfrom_CSV.shape

(7386, 17)

In [39]:
addrLUfrom_CSV.fillna(0, inplace = True)

In [40]:
model_file = pd.merge(model_file, addrLUfrom_CSV, left_on = 'addr_group', right_on = 'addr_group', suffixes= ("_x","_addr_group_cnt"))

In [41]:
model_file.shape

(89334, 1451)

In [42]:
model_file[0:12]

,Unnamed: 0,AV_BLDG,AV_LAND,GIS_ID,GROSS_AREA,LAND_SF,LIVING_ARE,LU,NUM_FLOORS,OWN_OCC,...,Condominium Parking,Exempt,Exempt 121A,Industrial,Mixed Use Res/Comm,Residential Land,Residential Single Family,Residential Three Family,Residential Two Family,XX
0,0,235300,129900,100001000,3353.0,1150.0,2202.0,Residential Three Family,3.0,1,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
1,51,56000,114600,100052000,2478.0,1618.0,1658.0,Residential Single Family,2.0,0,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
2,52,156900,122700,100053000,2730.0,1864.0,1816.0,Residential Single Family,2.0,1,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
3,53,163700,137500,100054000,2964.0,1640.0,2190.0,Residential Three Family,3.0,1,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
4,54,227700,137200,100055000,2972.0,1640.0,2088.0,Residential Three Family,3.0,0,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
5,2600,206400,112800,102991000,3344.0,1250.0,2251.0,Residential Two Family,2.5,1,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
6,2601,148900,97100,102992000,2330.0,1050.0,1441.0,Residential Single Family,2.0,1,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
7,2602,142300,97400,102993000,1984.0,1050.0,1232.0,Residential Single Family,2.0,1,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
8,2603,281800,131000,102994000,4005.0,1450.0,2650.0,Residential Three Family,2.5,0,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0
9,5909,157300,112400,106670000,2720.0,1575.0,1488.0,Residential Single Family,2.0,1,...,0.0,0.0,0.0,0.0,5.0,0.0,7.0,28.0,7.0,0.0


In [43]:
##
# Address group level:
model_file['Resl_Tax_parcels_per_addr'] = model_file['Residential Single Family'] + model_file['Residential Two Family'] + model_file['Residential Three Family'] + model_file['Apartments 4-6 Units'] + model_file['Apartments 7 Units above'] # +model_file['Residential Condo Unit']
model_file['Comml_Tax_parcels_per_addr'] = model_file['Commercial'] + model_file['Commercial Land'] + model_file[ 'Mixed Use Res/Comm'] + model_file['Commercial Condo Unit'] + model_file['Industrial']
model_file['OTHER_Tax_parcels_per_addr'] = model_file['Residential Land'] + model_file['Agricultural/Horticultural'] + model_file['Exempt']
model_file['ALL_Tax_parcels_per_addr'] = model_file['Resl_Tax_parcels_per_addr'] + model_file['Comml_Tax_parcels_per_addr'] + model_file['OTHER_Tax_parcels_per_addr']

# ##
# # Qtr-mile group level:
# model_file['Resl_Tax_parcels_per_Qmil'] = model_file['LU__Residential Single Family_FOR_Qmil_cnt'] + model_file['LU__Residential Two Family_FOR_Qmil_cnt'] + model_file['LU__Residential Three Family_FOR_Qmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_Qmil_cnt'] + model_file['LU__Apartments 7 Units above_FOR_Qmil_cnt'] + model_file['LU__Residential Condo Unit_FOR_Qmil_cnt']
# model_file['Comml_Tax_parcels_per_Qmil'] = model_file['LU__Commercial_FOR_Qmil_cnt'] + model_file['LU__Commercial Land_FOR_Qmil_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_Qmil_cnt'] + model_file['LU__Commercial Condo Unit_FOR_Qmil_cnt'] + model_file['LU__Industrial_FOR_Qmil_cnt']
# model_file['OTHER_Tax_parcels_per_Qmil'] = model_file['LU__Residential Land_FOR_Qmil_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_Qmil_cnt'] 
# model_file['ALL_Tax_parcels_per_Qmil'] = model_file['Resl_Tax_parcels_per_Qmil'] + model_file['Comml_Tax_parcels_per_Qmil'] + model_file['OTHER_Tax_parcels_per_Qmil']

# ##
# # Half-mile group level:
# model_file['Resl_Tax_parcels_per_hmil'] = model_file['LU__Residential Single Family_FOR_hmil_cnt'] + model_file['LU__Residential Two Family_FOR_hmil_cnt'] + model_file['LU__Residential Three Family_FOR_hmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] + model_file['LU__Apartments 7 Units above_FOR_hmil_cnt'] + model_file['LU__Residential Condo Unit_FOR_hmil_cnt']
# model_file['Comml_Tax_parcels_per_hmil'] = model_file['LU__Commercial_FOR_hmil_cnt'] + model_file['LU__Commercial Land_FOR_hmil_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_hmil_cnt'] + model_file['LU__Commercial Condo Unit_FOR_hmil_cnt'] + model_file['LU__Industrial_FOR_hmil_cnt']
# model_file['OTHER_Tax_parcels_per_hmil'] = model_file['LU__Residential Land_FOR_hmil_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_hmil_cnt'] 
# model_file['ALL_Tax_parcels_per_hmil'] = model_file['Resl_Tax_parcels_per_hmil'] + model_file['Comml_Tax_parcels_per_hmil'] + model_file['OTHER_Tax_parcels_per_hmil']

# ##
# # Neighborhood group level:
# model_file['Resl_Tax_parcels_per_nbhd'] = model_file['LU__Residential Single Family_FOR_nbhd_cnt'] + model_file['LU__Residential Two Family_FOR_nbhd_cnt'] + model_file['LU__Residential Three Family_FOR_nbhd_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] + model_file['LU__Apartments 7 Units above_FOR_nbhd_cnt'] + model_file['LU__Residential Condo Unit_FOR_nbhd_cnt']
# model_file['Comml_Tax_parcels_per_nbhd'] = model_file['LU__Commercial_FOR_nbhd_cnt'] + model_file['LU__Commercial Land_FOR_nbhd_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_nbhd_cnt'] + model_file['LU__Commercial Condo Unit_FOR_nbhd_cnt'] + model_file['LU__Industrial_FOR_nbhd_cnt']
# model_file['OTHER_Tax_parcels_per_nbhd'] = model_file['LU__Residential Land_FOR_nbhd_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_nbhd_cnt'] 
# model_file['ALL_Tax_parcels_per_nbhd'] = model_file['Resl_Tax_parcels_per_nbhd'] + model_file['Comml_Tax_parcels_per_nbhd'] + model_file['OTHER_Tax_parcels_per_nbhd']

# ##
# # Census Tract level:
# model_file['Resl_Tax_parcels_per_ctrt'] = model_file['LU__Residential Single Family_FOR_ctrt_cnt'] + model_file['LU__Residential Two Family_FOR_ctrt_cnt'] + model_file['LU__Residential Three Family_FOR_ctrt_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] + model_file['LU__Apartments 7 Units above_FOR_ctrt_cnt'] + model_file['LU__Residential Condo Unit_FOR_ctrt_cnt']
# model_file['Comml_Tax_parcels_per_ctrt'] = model_file['LU__Commercial_FOR_ctrt_cnt'] + model_file['LU__Commercial Land_FOR_ctrt_cnt'] + model_file[ 'LU__Mixed Use Res/Comm_FOR_ctrt_cnt'] + model_file['LU__Commercial Condo Unit_FOR_ctrt_cnt'] + model_file['LU__Industrial_FOR_ctrt_cnt']
# model_file['OTHER_Tax_parcels_per_ctrt'] = model_file['LU__Residential Land_FOR_ctrt_cnt'] + model_file['LU__Agricultural/Horticultural'] + model_file['LU__Exempt_FOR_ctrt_cnt'] 
# model_file['ALL_Tax_parcels_per_ctrt'] = model_file['Resl_Tax_parcels_per_ctrt'] + model_file['Comml_Tax_parcels_per_ctrt'] + model_file['OTHER_Tax_parcels_per_ctrt']

In [44]:
##
# Address group level:

model_file['LU_Single_Family_pct_of_rsl_addr'] = model_file['Residential Single Family'] / model_file['Resl_Tax_parcels_per_addr']
model_file['LU_Three_Six_family_pct_of_rsl_addr'] = (model_file['Residential Two Family'] + model_file['Residential Three Family'] + model_file['Apartments 4-6 Units'] ) / model_file['Resl_Tax_parcels_per_addr']
model_file['LU_Two_Family_pct_of_rsl_addr'] = model_file['Residential Two Family'] / model_file['Resl_Tax_parcels_per_addr']
model_file['LU_Three_Family_pct_of_rsl_addr'] = model_file['Residential Three Family'] / model_file['Resl_Tax_parcels_per_addr']
model_file['Apts_4to6_pct_of_rsl_addr'] = model_file['Apartments 4-6 Units'] / model_file['Resl_Tax_parcels_per_addr']
model_file['Apts_7plus_pct_of_rsl_addr'] = model_file['Apartments 7 Units above'] /model_file['Resl_Tax_parcels_per_addr']
#model_file['Resl_Condo_Unit_pct_of_rsl_addr'] = model_file['Residential Condo Unit'] /model_file['Resl_Tax_parcels_per_addr']
model_file['LU_ALL_Multi_housing_pct_of_addr'] = (model_file['Residential Two Family'] + model_file['Residential Three Family'] + model_file['Apartments 4-6 Units'] + model_file['Apartments 7 Units above'] ) / model_file['Resl_Tax_parcels_per_addr']
model_file['Mixed_Use_ResComml_pct_of_addr'] = model_file['Mixed Use Res/Comm'] / model_file['ALL_Tax_parcels_per_addr']
model_file['Comml_pct_of_addr'] = model_file['Comml_Tax_parcels_per_addr'] / model_file['ALL_Tax_parcels_per_addr'] 

# ##
# # Qtr-mile group level:
# model_file['LU_Single_Family_pct_of_rsl_Qmil'] = model_file['LU__Residential Single Family_FOR_Qmil_cnt'] / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU_Three_Six_family_pct_of_rsl_Qmil'] = (model_file['LU__Residential Two Family_FOR_Qmil_cnt'] + model_file['LU__Residential Three Family_FOR_Qmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_Qmil_cnt'] ) / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU_Two_Family_pct_of_rsl_Qmil'] = model_file['LU__Residential Two Family_FOR_Qmil_cnt'] / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU_Three_Family_pct_of_rsl_Qmil'] = model_file['LU__Residential Three Family_FOR_Qmil_cnt'] / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU__Apts_4to6_pct_of_rsl_Qmil'] = model_file['LU__Apartments 4-6 Units_FOR_Qmil_cnt'] / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU__Apts_7plus_pct_of_rsl_Qmil'] = model_file['LU__Apartments 7 Units above_FOR_Qmil_cnt'] / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU__Resl_Condo_Unit_pct_of_rsl_Qmil'] = model_file['LU__Residential Condo Unit_FOR_Qmil_cnt'] / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU_ALL_Multi_housing_pct_of_Qmil'] = (model_file['LU__Residential Two Family_FOR_Qmil_cnt'] + model_file['LU__Residential Three Family_FOR_Qmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_Qmil_cnt'] + model_file['LU__Apartments 7 Units above_FOR_Qmil_cnt'] + model_file['LU__Residential Condo Unit_FOR_Qmil_cnt']) / model_file['Resl_Tax_parcels_per_Qmil']
# model_file['LU__Mixed_Use_ResComml_pct_of_Qmil'] = model_file['LU__Mixed Use Res/Comm_FOR_Qmil_cnt'] / model_file['ALL_Tax_parcels_per_Qmil']
# model_file['LU__Comml_pct_of_Qmil'] = model_file['Comml_Tax_parcels_per_Qmil'] / model_file['ALL_Tax_parcels_per_Qmil'] 


# ##
# # Half-mile group level:
# model_file['LU_Single_Family_pct_of_rsl_hmil'] = model_file['LU__Residential Single Family_FOR_hmil_cnt'] / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU_Three_Six_family_pct_of_rsl_hmil'] = (model_file['LU__Residential Two Family_FOR_hmil_cnt'] + model_file['LU__Residential Three Family_FOR_hmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] ) / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU_Two_Family_pct_of_rsl_hmil'] = model_file['LU__Residential Two Family_FOR_hmil_cnt'] / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU_Three_Family_pct_of_rsl_hmil'] = model_file['LU__Residential Three Family_FOR_hmil_cnt'] / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU__Apts_4to6_pct_of_rsl_hmil'] = model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU__Apts_7plus_pct_of_rsl_hmil'] = model_file['LU__Apartments 7 Units above_FOR_hmil_cnt'] / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU__Resl_Condo_Unit_pct_of_rsl_hmil'] = model_file['LU__Residential Condo Unit_FOR_hmil_cnt'] / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU_ALL_Multi_housing_pct_of_hmil'] = (model_file['LU__Residential Two Family_FOR_hmil_cnt'] + model_file['LU__Residential Three Family_FOR_hmil_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_hmil_cnt'] + model_file['LU__Apartments 7 Units above_FOR_hmil_cnt'] + model_file['LU__Residential Condo Unit_FOR_hmil_cnt']) / model_file['Resl_Tax_parcels_per_hmil']
# model_file['LU__Mixed_Use_ResComml_pct_of_hmil'] = model_file['LU__Mixed Use Res/Comm_FOR_hmil_cnt'] / model_file['ALL_Tax_parcels_per_hmil']
# model_file['LU__Comml_pct_of_hmil'] = model_file['Comml_Tax_parcels_per_hmil'] / model_file['ALL_Tax_parcels_per_hmil'] 

# ##
# # Neighborhood group level:
# model_file['LU_Single_Family_pct_of_rsl_nbhd'] = model_file['LU__Residential Single Family_FOR_nbhd_cnt'] / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU_Three_Six_family_pct_of_rsl_nbhd'] = (model_file['LU__Residential Two Family_FOR_nbhd_cnt'] + model_file['LU__Residential Three Family_FOR_nbhd_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] ) / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU_Two_Family_pct_of_rsl_nbhd'] = model_file['LU__Residential Two Family_FOR_nbhd_cnt'] / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU_Three_Family_pct_of_rsl_nbhd'] = model_file['LU__Residential Three Family_FOR_nbhd_cnt'] / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU__Apts_4to6_pct_of_rsl_nbhd'] = model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU__Apts_7plus_pct_of_rsl_nbhd'] = model_file['LU__Apartments 7 Units above_FOR_nbhd_cnt'] / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU__Resl_Condo_Unit_pct_of_rsl_nbhd'] = model_file['LU__Residential Condo Unit_FOR_nbhd_cnt'] / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU_ALL_Multi_housing_pct_of_nbhd'] = (model_file['LU__Residential Two Family_FOR_nbhd_cnt'] + model_file['LU__Residential Three Family_FOR_nbhd_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_nbhd_cnt'] + model_file['LU__Apartments 7 Units above_FOR_nbhd_cnt'] + model_file['LU__Residential Condo Unit_FOR_nbhd_cnt']) / model_file['Resl_Tax_parcels_per_nbhd']
# model_file['LU__Mixed_Use_ResComml_pct_of_nbhd'] = model_file['LU__Mixed Use Res/Comm_FOR_nbhd_cnt'] / model_file['ALL_Tax_parcels_per_nbhd']
# model_file['LU__Comml_pct_of_nbhd'] = model_file['Comml_Tax_parcels_per_nbhd'] / model_file['ALL_Tax_parcels_per_nbhd'] 

# ## 
# # Census tract level:
# model_file['LU_Single_Family_pct_of_rsl_ctrt'] = model_file['LU__Residential Single Family_FOR_ctrt_cnt'] / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU_Three_Six_family_pct_of_rsl_ctrt'] = (model_file['LU__Residential Two Family_FOR_ctrt_cnt'] + model_file['LU__Residential Three Family_FOR_ctrt_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] ) / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU_Two_Family_pct_of_rsl_ctrt'] = model_file['LU__Residential Two Family_FOR_ctrt_cnt'] / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU_Three_Family_pct_of_rsl_ctrt'] = model_file['LU__Residential Three Family_FOR_ctrt_cnt'] / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU__Apts_4to6_pct_of_rsl_ctrt'] = model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU__Apts_7plus_pct_of_rsl_ctrt'] = model_file['LU__Apartments 7 Units above_FOR_ctrt_cnt'] / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU__Resl_Condo_Unit_pct_of_rsl_ctrt'] = model_file['LU__Residential Condo Unit_FOR_ctrt_cnt'] / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU_ALL_Multi_housing_pct_of_ctrt'] = (model_file['LU__Residential Two Family_FOR_ctrt_cnt'] + model_file['LU__Residential Three Family_FOR_ctrt_cnt'] + model_file['LU__Apartments 4-6 Units_FOR_ctrt_cnt'] + model_file['LU__Apartments 7 Units above_FOR_ctrt_cnt'] + model_file['LU__Residential Condo Unit_FOR_ctrt_cnt']) / model_file['Resl_Tax_parcels_per_ctrt']
# model_file['LU__Mixed_Use_ResComml_pct_of_ctrt'] = model_file['LU__Mixed Use Res/Comm_FOR_ctrt_cnt'] / model_file['ALL_Tax_parcels_per_ctrt']
# model_file['LU__Comml_pct_of_ctrt'] = model_file['Comml_Tax_parcels_per_ctrt'] / model_file['ALL_Tax_parcels_per_ctrt']


In [45]:
## 
# Ratio of TOTAL ROOMS
##
model_file['R_TOTAL_RM_VS_addr_gr_AVG'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_addr_avg']
model_file['R_TOTAL_RM_VS_Qmil_gr_AVG'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_Qmil_avg']
model_file['R_TOTAL_RM_VS_hmil_gr_AVG'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_hmil_avg']
model_file['R_TOTAL_RM_VS_nbhd_gr_AVG'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_nbhd_avg']
model_file['R_TOTAL_RM_VS_ctrt_gr_AVG'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_ctrt_avg']

# #model_file['R_TOTAL_RM_VS_addr_gr_MED'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_addr_med']
# model_file['R_TOTAL_RM_VS_Qmil_gr_MED'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_Qmil_med']
# model_file['R_TOTAL_RM_VS_hmil_gr_MED'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_hmil_med']
# model_file['R_TOTAL_RM_VS_nbhd_gr_MED'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_nbhd_med']
# model_file['R_TOTAL_RM_VS_ctrt_gr_MED'] = model_file['R_TOTAL_RM'] / model_file['R_TOTAL_RM_FOR_ctrt_med']

## 
# Ratio of LIVING AREA
##
model_file['LIVING_ARE_VS_addr_gr_AVG'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_addr_avg']
model_file['LIVING_ARE_VS_Qmil_gr_AVG'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_Qmil_avg']
model_file['LIVING_ARE_VS_hmil_gr_AVG'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_hmil_avg']
model_file['LIVING_ARE_VS_nbhd_gr_AVG'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_nbhd_avg']
model_file['LIVING_ARE_VS_ctrt_gr_AVG'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_ctrt_avg']

# model_file['LIVING_ARE_VS_addr_gr_MED'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_addr_med']
# model_file['LIVING_ARE_VS_Qmil_gr_MED'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_Qmil_med']
# model_file['LIVING_ARE_VS_hmil_gr_MED'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_hmil_med']
# model_file['LIVING_ARE_VS_nbhd_gr_MED'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_nbhd_med']
# model_file['LIVING_ARE_VS_ctrt_gr_MED'] = model_file['LIVING_ARE'] / model_file['LIVING_ARE_FOR_ctrt_med']


## 
# Ratio of NUM FLOORS
##
model_file['NUM_FLOORS_VS_addr_gr_AVG'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_addr_avg']
model_file['NUM_FLOORS_VS_Qmil_gr_AVG'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_Qmil_avg']
model_file['NUM_FLOORS_VS_hmil_gr_AVG'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_hmil_avg']
model_file['NUM_FLOORS_VS_nbhd_gr_AVG'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_nbhd_avg']
model_file['NUM_FLOORS_VS_ctrt_gr_AVG'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_ctrt_avg']

# model_file['NUM_FLOORS_VS_addr_gr_MED'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_addr_med']
# model_file['NUM_FLOORS_VS_Qmil_gr_MED'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_Qmil_med']
# model_file['NUM_FLOORS_VS_hmil_gr_MED'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_hmil_med']
# model_file['NUM_FLOORS_VS_nbhd_gr_MED'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_nbhd_med']
# model_file['NUM_FLOORS_VS_ctrt_gr_MED'] = model_file['NUM_FLOORS'] / model_file['NUM_FLOORS_FOR_ctrt_med']


## 
# Ratio of Heat Type Electric
##
model_file['Heat_Type__Electric_VS_addr_gr_AVG'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_addr_avg']
model_file['Heat_Type__Electric_VS_Qmil_gr_AVG'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_Qmil_avg']
model_file['Heat_Type__Electric_VS_hmil_gr_AVG'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_hmil_avg']
model_file['Heat_Type__Electric_VS_nbhd_gr_AVG'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_nbhd_avg']
model_file['Heat_Type__Electric_VS_ctrt_gr_AVG'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_ctrt_avg']

# model_file['Heat_Type__Electric_VS_addr_gr_MED'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_addr_med']
# model_file['Heat_Type__Electric_VS_Qmil_gr_MED'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_Qmil_med']
# model_file['Heat_Type__Electric_VS_hmil_gr_MED'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_hmil_med']
# model_file['Heat_Type__Electric_VS_nbhd_gr_MED'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_nbhd_med']
# model_file['Heat_Type__Electric_VS_ctrt_gr_MED'] = model_file['Heat_Type__Electric'] / model_file['Heat_Type__Electric_FOR_ctrt_med']

## 
# Ratio of Thr12011to2013_excluded
##
model_file['Thr12011to2013_excluded_VS_addr_gr_AVG'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_addr_avg']
model_file['Thr12011to2013_excluded_VS_Qmil_gr_AVG'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_Qmil_avg']
model_file['Thr12011to2013_excluded_VS_hmil_gr_AVG'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_hmil_avg']
model_file['Thr12011to2013_excluded_VS_nbhd_gr_AVG'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_nbhd_avg']
model_file['Thr12011to2013_excluded_VS_ctrt_gr_AVG'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_ctrt_avg']

# model_file['Thr12011to2013_excluded_VS_addr_gr_MED'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_addr_med']
# model_file['Thr12011to2013_excluded_VS_Qmil_gr_MED'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_Qmil_med']
# model_file['Thr12011to2013_excluded_VS_hmil_gr_MED'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_hmil_med']
# model_file['Thr12011to2013_excluded_VS_nbhd_gr_MED'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_nbhd_med']
# model_file['Thr12011to2013_excluded_VS_ctrt_gr_MED'] = model_file['Thr12011to2013_excluded'] / model_file['Thr12011to2013_excluded_FOR_ctrt_med']

## 
# Ratio of Thr12014to2015_excluded
##
model_file['Thr12014to2015_excluded_VS_addr_gr_AVG'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_addr_avg']
model_file['Thr12014to2015_excluded_VS_Qmil_gr_AVG'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_Qmil_avg']
model_file['Thr12014to2015_excluded_VS_hmil_gr_AVG'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_hmil_avg']
model_file['Thr12014to2015_excluded_VS_nbhd_gr_AVG'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_nbhd_avg']
model_file['Thr12014to2015_excluded_VS_ctrt_gr_AVG'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_ctrt_avg']

# model_file['Thr12014to2015_excluded_VS_addr_gr_MED'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_addr_med']
# model_file['Thr12014to2015_excluded_VS_Qmil_gr_MED'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_Qmil_med']
# model_file['Thr12014to2015_excluded_VS_hmil_gr_MED'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_hmil_med']
# model_file['Thr12014to2015_excluded_VS_nbhd_gr_MED'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_nbhd_med']
# model_file['Thr12014to2015_excluded_VS_ctrt_gr_MED'] = model_file['Thr12014to2015_excluded'] / model_file['Thr12014to2015_excluded_FOR_ctrt_med']

## 
# Ratio of Thr12015_excluded
##
model_file['Thr12015_excluded_VS_addr_gr_AVG'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_addr_avg']
model_file['Thr12015_excluded_VS_Qmil_gr_AVG'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_Qmil_avg']
model_file['Thr12015_excluded_VS_hmil_gr_AVG'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_hmil_avg']
model_file['Thr12015_excluded_VS_nbhd_gr_AVG'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_nbhd_avg']
model_file['Thr12015_excluded_VS_ctrt_gr_AVG'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_ctrt_avg']

# model_file['Thr12015_excluded_VS_addr_gr_MED'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_addr_med']
# model_file['Thr12015_excluded_VS_Qmil_gr_MED'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_Qmil_med']
# model_file['Thr12015_excluded_VS_hmil_gr_MED'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_hmil_med']
# model_file['Thr12015_excluded_VS_nbhd_gr_MED'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_nbhd_med']
# model_file['Thr12015_excluded_VS_ctrt_gr_MED'] = model_file['Thr12015_excluded'] / model_file['Thr12015_excluded_FOR_ctrt_med']

## 
# Ratio of Thr12014to2015_Building
##
model_file['Thr12014to2015_Building_VS_addr_gr_AVG'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_addr_avg']
model_file['Thr12014to2015_Building_VS_Qmil_gr_AVG'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_Qmil_avg']
model_file['Thr12014to2015_Building_VS_hmil_gr_AVG'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_hmil_avg']
model_file['Thr12014to2015_Building_VS_nbhd_gr_AVG'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_nbhd_avg']
model_file['Thr12014to2015_Building_VS_ctrt_gr_AVG'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_ctrt_avg']

# model_file['Thr12014to2015_Building_VS_addr_gr_MED'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_addr_med']
# model_file['Thr12014to2015_Building_VS_Qmil_gr_MED'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_Qmil_med']
# model_file['Thr12014to2015_Building_VS_hmil_gr_MED'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_hmil_med']
# model_file['Thr12014to2015_Building_VS_nbhd_gr_MED'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_nbhd_med']
# model_file['Thr12014to2015_Building_VS_ctrt_gr_MED'] = model_file['Thr12014to2015_Building'] / model_file['Thr12014to2015_Building_FOR_ctrt_med']

## 
# Ratio of Thr12014to2015_Recycling-Pickup
##
model_file['Thr12014to2015_Recycling-Pickup_VS_addr_gr_AVG'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_addr_avg']
model_file['Thr12014to2015_Recycling-Pickup_VS_Qmil_gr_AVG'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_Qmil_avg']
model_file['Thr12014to2015_Recycling-Pickup_VS_hmil_gr_AVG'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_hmil_avg']
model_file['Thr12014to2015_Recycling-Pickup_VS_nbhd_gr_AVG'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg']
model_file['Thr12014to2015_Recycling-Pickup_VS_ctrt_gr_AVG'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg']

# model_file['Thr12014to2015_Recycling-Pickup_VS_addr_gr_MED'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_addr_med']
# model_file['Thr12014to2015_Recycling-Pickup_VS_Qmil_gr_MED'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_Qmil_med']
# model_file['Thr12014to2015_Recycling-Pickup_VS_hmil_gr_MED'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_hmil_med']
# model_file['Thr12014to2015_Recycling-Pickup_VS_nbhd_gr_MED'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_nbhd_med']
# model_file['Thr12014to2015_Recycling-Pickup_VS_ctrt_gr_MED'] = model_file['Thr12014to2015_Recycling-Pickup'] / model_file['Thr12014to2015_Recycling-Pickup_FOR_ctrt_med']
## 
# Ratio of Shape vs. Len
##
model_file['shp_v_len_VS_addr_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_addr_avg']
model_file['shp_v_len_VS_Qmil_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_Qmil_avg']
model_file['shp_v_len_VS_hmil_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_hmil_avg']
model_file['shp_v_len_VS_nbhd_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_nbhd_avg']
model_file['shp_v_len_VS_ctrt_gr_AVG'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_ctrt_avg']

model_file['shp_v_len_VS_addr_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_addr_med']
model_file['shp_v_len_VS_Qmil_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_Qmil_med']
model_file['shp_v_len_VS_hmil_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_hmil_med']
model_file['shp_v_len_VS_nbhd_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_nbhd_med']
model_file['shp_v_len_VS_ctrt_gr_MED'] = model_file['shape_vs_length'] / model_file['shape_vs_length_FOR_ctrt_med']

##
#  Age of home in 2016
##
model_file['YR_BUILT_VS_addr_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_addr_avg']
model_file['YR_BUILT_VS_Qmil_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_Qmil_avg']
model_file['YR_BUILT_VS_hmil_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_hmil_avg']
model_file['YR_BUILT_VS_nbhd_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_nbhd_avg']
model_file['YR_BUILT_VS_ctrt_gr_AVG'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_ctrt_avg']

model_file['YR_BUILT_VS_addr_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_addr_med']
model_file['YR_BUILT_VS_Qmil_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_Qmil_med']
#model_file['YR_BUILT_VS_hmil_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_hmil_med'] (had inf value)
model_file['YR_BUILT_VS_nbhd_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_nbhd_med']
model_file['YR_BUILT_VS_ctrt_gr_MED'] = model_file['YR_BUILT'] / model_file['YR_BUILT_FOR_ctrt_med']

##
# GROSS AREA
#
model_file['GROSS_AREA_VS_addr_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_addr_avg']
model_file['GROSS_AREA_VS_Qmil_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_Qmil_avg']
model_file['GROSS_AREA_VS_hmil_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_hmil_avg']
model_file['GROSS_AREA_VS_nbhd_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_nbhd_avg']
model_file['GROSS_AREA_VS_ctrt_gr_AVG'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_ctrt_avg']

model_file['GROSS_AREA_VS_addr_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_addr_med']
model_file['GROSS_AREA_VS_Qmil_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_Qmil_med']
#model_file['GROSS_AREA_VS_hmil_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_hmil_med'](had inf value)
model_file['GROSS_AREA_VS_nbhd_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_nbhd_med']
model_file['GROSS_AREA_VS_ctrt_gr_MED'] = model_file['GROSS_AREA'] / model_file['GROSS_AREA_FOR_ctrt_med']

##
# 311 recycling related calls
##

model_file['Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_addr_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_Qmil_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_Qmil_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_hmil_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg']
model_file['Thr12011to2013_Recycling-Pickup_VS_addr_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_addr_med']
model_file['Thr12011to2013_Recycling-Pickup_VS_Qmil_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_Qmil_med']
#model_file['Thr12011to2013_Recycling-Pickup_VS_hmil_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_hmil_med']
model_file['Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_nbhd_med']
model_file['Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_MED'] = model_file['Thr12011to2013_Recycling-Pickup'] / model_file['Thr12011to2013_Recycling-Pickup_FOR_ctrt_med']



## Create 'drop' lists for year-specific files

Basically, a year-specific file will not include any data that occurs in the future

In [46]:
YR_2016_DROPS =  ['Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
# YR_2015_DROPS = [ 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
# YR_2014_DROPS = ['Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
# YR_2013_DROPS = ['Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med', 'Top2013PermTypdollars', 'Top2013PermTypNum', 'Thr12013_excluded', 'Thr12013_Enviro', 'Thr12013_Building', 'Thr12013_Street-Sidewalk', 'Thr12013_Recycling-Pickup', 'Thr12013_Trees', 'Thr12013_Graffiti', 'Thr12013_Housing', 'NOT_Top2013PermTypdollars', 'NOT_Top2013PermTypNum', 'NOT_Top2013PermTypNum_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_addr_avg', 'NOT_Top2013PermTypNum_FOR_addr_avg', 'NOT_Top2013PermTypdollars_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_nbhd_avg', 'NOT_Top2013PermTypdollars_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_nbhd_avg', 'NOT_Top2013PermTypNum_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2013PermTypNum_FOR_ctrt_cnt', 'NOT_Top2013PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2013PermTypNum_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_nbhd_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_med', 'NOT_Top2013PermTypNum_FOR_hmil_med', 'NOT_Top2013PermTypdollars_FOR_nbhd_med', 'NOT_Top2013PermTypNum_FOR_nbhd_med', 'NOT_Top2013PermTypdollars_FOR_ctrt_med', 'NOT_Top2013PermTypNum_FOR_ctrt_med', 'Top2014PermTypdollars', 'Top2014PermTypNum', 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
# YR_2012_COL_DROPS = ['Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med','Top2012PermTypdollars', 'Top2012PermTypNum', 'Thr12012_Street-Sidewalk', 'Thr12012_Recycling-Pickup', 'Thr12012_excluded', 'Thr12012_Enviro', 'Thr12012_Trees', 'Thr12012_Building', 'Thr12012_Graffiti', 'Thr12012_Housing', 'NOT_Top2012PermTypdollars', 'NOT_Top2012PermTypNum', 'NOT_Top2012PermTypdollars_FOR_hmil_avg', 'NOT_Top2012PermTypNum_FOR_hmil_avg', 'NOT_Top2012PermTypdollars_FOR_addr_avg', 'NOT_Top2012PermTypNum_FOR_addr_avg', 'NOT_Top2012PermTypdollars_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_hmil_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_hmil_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_hmil_med', 'NOT_Top2012PermTypNum_FOR_hmil_med', 'NOT_Top2012PermTypdollars_FOR_nbhd_med', 'NOT_Top2012PermTypNum_FOR_nbhd_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_med', 'NOT_Top2012PermTypNum_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med', 'Top2013PermTypdollars', 'Top2013PermTypNum', 'Thr12013_excluded', 'Thr12013_Enviro', 'Thr12013_Building', 'Thr12013_Street-Sidewalk', 'Thr12013_Recycling-Pickup', 'Thr12013_Trees', 'Thr12013_Graffiti', 'Thr12013_Housing', 'NOT_Top2013PermTypdollars', 'NOT_Top2013PermTypNum', 'NOT_Top2013PermTypNum_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_addr_avg', 'NOT_Top2013PermTypNum_FOR_addr_avg', 'NOT_Top2013PermTypdollars_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_nbhd_avg', 'NOT_Top2013PermTypdollars_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_nbhd_avg', 'NOT_Top2013PermTypNum_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2013PermTypNum_FOR_ctrt_cnt', 'NOT_Top2013PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2013PermTypNum_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_nbhd_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_med', 'NOT_Top2013PermTypNum_FOR_hmil_med', 'NOT_Top2013PermTypdollars_FOR_nbhd_med', 'NOT_Top2013PermTypNum_FOR_nbhd_med', 'NOT_Top2013PermTypdollars_FOR_ctrt_med', 'NOT_Top2013PermTypNum_FOR_ctrt_med', 'Top2014PermTypdollars', 'Top2014PermTypNum', 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
# YR_2011_COL_DROPS = ['Top2011PermTypdollars', 'Top2011PermTypNum', 'Thr12011_excluded', 'Thr12011_Recycling-Pickup', 'Thr12011to2013_Enviro', 'Thr12011_Trees', 'Thr12011_Enviro', 'Thr12011_Building', 'Thr12011_Street-Sidewalk', 'Thr12011_Graffiti', 'Thr12011_Housing', 'NOT_Top2011PermTypdollars', 'NOT_Top2011PermTypdollars_FOR_addr_avg', 'NOT_Top2011PermTypdollars_FOR_hmil_avg', 'NOT_Top2011PermTypdollars_FOR_ctrt_avg', 'NOT_Top2011PermTypdollars_FOR_nbhd_avg', 'NOT_Top2011PermTypNum', 'NOT_Top2011PermTypdollars_FOR_addr_cnt', 'NOT_Top2011PermTypdollars_FOR_hmil_cnt', 'NOT_Top2011PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2011PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2011PermTypdollars_FOR_addr_med', 'NOT_Top2011PermTypdollars_FOR_hmil_med', 'NOT_Top2011PermTypdollars_FOR_nbhd_med', 'NOT_Top2011PermTypdollars_FOR_ctrt_med', 'Top2012PermTypdollars', 'Top2012PermTypNum', 'Thr12012_Street-Sidewalk', 'Thr12012_Recycling-Pickup', 'Thr12012_excluded', 'Thr12012_Enviro', 'Thr12012_Trees', 'Thr12012_Building', 'Thr12012_Graffiti', 'Thr12012_Housing', 'NOT_Top2012PermTypdollars', 'NOT_Top2012PermTypNum', 'NOT_Top2012PermTypdollars_FOR_hmil_avg', 'NOT_Top2012PermTypNum_FOR_hmil_avg', 'NOT_Top2012PermTypdollars_FOR_addr_avg', 'NOT_Top2012PermTypNum_FOR_addr_avg', 'NOT_Top2012PermTypdollars_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_avg', 'NOT_Top2012PermTypNum_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_cnt', 'NOT_Top2012PermTypdollars_FOR_hmil_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_hmil_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_nbhd_avg', 'NOT_Top2012PermTypNum_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2012PermTypNum_FOR_nbhd_cnt', 'NOT_Top2012PermTypdollars_FOR_addr_med', 'NOT_Top2012PermTypdollars_FOR_hmil_med', 'NOT_Top2012PermTypNum_FOR_hmil_med', 'NOT_Top2012PermTypdollars_FOR_nbhd_med', 'NOT_Top2012PermTypNum_FOR_nbhd_med', 'NOT_Top2012PermTypdollars_FOR_ctrt_med', 'NOT_Top2012PermTypNum_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_addr_avg', 'Thr12011to2013_Recycling-Pickup_VS_addr_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_AVG', 'Thr12011to2013_Recycling-Pickup_VS_hmil_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_avg', 'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_AVG', 'Thr12011to2013_Recycling-Pickup_FOR_addr_cnt', 'Thr12011to2013_excluded', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_avg', 'Thr12011to2013_Recycling-Pickup', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_cnt', 'Thr12011to2013_Recycling-Pickup_FOR_addr_med', 'Thr12011to2013_Street-Sidewalk', 'Thr12011to2013_Recycling-Pickup_FOR_hmil_med', 'Thr12011to2013_Trees', 'Thr12011to2013_Building', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12011to2013_Graffiti', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_avg', 'Thr12011to2013_Recycling-Pickup_FOR_ctrt_med', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12011to2013_Housing', 'Thr12011to2013_Recycling-Pickup_FOR_nbhd_med', 'Top2013PermTypdollars', 'Top2013PermTypNum', 'Thr12013_excluded', 'Thr12013_Enviro', 'Thr12013_Building', 'Thr12013_Street-Sidewalk', 'Thr12013_Recycling-Pickup', 'Thr12013_Trees', 'Thr12013_Graffiti', 'Thr12013_Housing', 'NOT_Top2013PermTypdollars', 'NOT_Top2013PermTypNum', 'NOT_Top2013PermTypNum_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_addr_avg', 'NOT_Top2013PermTypNum_FOR_addr_avg', 'NOT_Top2013PermTypdollars_FOR_hmil_avg', 'NOT_Top2013PermTypdollars_FOR_nbhd_avg', 'NOT_Top2013PermTypdollars_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_ctrt_avg', 'NOT_Top2013PermTypNum_FOR_nbhd_avg', 'NOT_Top2013PermTypNum_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_cnt', 'NOT_Top2013PermTypdollars_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_hmil_cnt', 'NOT_Top2013PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2013PermTypNum_FOR_ctrt_cnt', 'NOT_Top2013PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2013PermTypNum_FOR_addr_med', 'NOT_Top2013PermTypNum_FOR_nbhd_cnt', 'NOT_Top2013PermTypdollars_FOR_hmil_med', 'NOT_Top2013PermTypNum_FOR_hmil_med', 'NOT_Top2013PermTypdollars_FOR_nbhd_med', 'NOT_Top2013PermTypNum_FOR_nbhd_med', 'NOT_Top2013PermTypdollars_FOR_ctrt_med', 'NOT_Top2013PermTypNum_FOR_ctrt_med', 'Top2014PermTypdollars', 'Top2014PermTypNum', 'Thr12014to2015_Recycling-Pickup_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_avg', 'Thr12014to2015_Building', 'Thr12014to2015_excluded_FOR_addr_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_avg', 'Thr12014to2015_excluded_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_avg', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_avg', 'Thr12014to2015_excluded_FOR_addr_cnt', 'Thr12014to2015_excluded', 'Thr12014_excluded', 'Thr12014to2015_Recycling-Pickup_FOR_addr_cnt', 'Thr12014_Enviro', 'Thr12014to2015_excluded_FOR_ctrt_avg', 'Thr12014to2015_Street-Sidewalk_FOR_addr_cnt', 'Thr12014to2015_Enviro', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_avg', 'Thr12014to2015_Recycling-Pickup', 'Thr12014to2015_excluded_FOR_hmil_cnt', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_cnt', 'Thr12014_Recycling-Pickup', 'Thr12014to2015_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_addr_med', 'Thr12014_Street-Sidewalk', 'Thr12014to2015_excluded_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_cnt', 'Thr12014to2015_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_addr_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_cnt', 'Thr12014to2015_Street-Sidewalk_FOR_addr_med', 'Thr12014to2015_excluded_FOR_nbhd_avg', 'Thr12014to2015_Trees', 'Thr12014_Building', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_avg', 'Thr12014to2015_excluded_FOR_hmil_med', 'Thr12014_Trees', 'Thr12014to2015_Housing', 'Thr12014to2015_Recycling-Pickup_FOR_hmil_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_avg', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_cnt', 'Thr12014_Graffiti', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_cnt', 'Thr12014to2015_excluded_FOR_nbhd_med', 'Thr12014to2015_excluded_FOR_nbhd_cnt', 'Thr12014_Housing', 'Thr12014to2015_Street-Sidewalk_FOR_hmil_med', 'Thr12014to2015_Recycling-Pickup_FOR_nbhd_med', 'Thr12014to2015_Street-Sidewalk_FOR_nbhd_med', 'Thr12014to2015_Recycling-Pickup_FOR_ctrt_med', 'Thr12014to2015_Street-Sidewalk_FOR_ctrt_med', 'Thr12014to2015_excluded_FOR_ctrt_med', 'NOT_Top2014PermTypdollars', 'NOT_Top2014PermTypNum', 'NOT_Top2014PermTypdollars_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_addr_avg', 'NOT_Top2014PermTypNum_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_hmil_avg', 'NOT_Top2014PermTypdollars_FOR_ctrt_avg', 'NOT_Top2014PermTypdollars_FOR_nbhd_avg', 'NOT_Top2014PermTypNum_FOR_ctrt_avg', 'NOT_Top2014PermTypNum_FOR_nbhd_avg', 'NOT_Top2014PermTypdollars_FOR_addr_cnt', 'NOT_Top2014PermTypdollars_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_addr_cnt', 'NOT_Top2014PermTypNum_FOR_hmil_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_cnt', 'NOT_Top2014PermTypNum_FOR_addr_med', 'NOT_Top2014PermTypNum_FOR_ctrt_cnt', 'NOT_Top2014PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2014PermTypNum_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2014PermTypdollars_FOR_hmil_med', 'NOT_Top2014PermTypNum_FOR_hmil_med', 'NOT_Top2014PermTypdollars_FOR_nbhd_med', 'NOT_Top2014PermTypNum_FOR_nbhd_med', 'NOT_Top2014PermTypdollars_FOR_ctrt_med', 'NOT_Top2014PermTypNum_FOR_ctrt_med', 'Top2015PermTypdollars', 'Top2015PermTypNum', 'Thr12015_Building', 'Thr12015_excluded', 'Thr12015_Recycling-Pickup', 'Thr12015_Street-Sidewalk', 'Thr12015_Enviro', 'Thr12015_Trees', 'Thr12015_Graffiti', 'Thr12015_Housing', 'NOT_Top2015PermTypdollars', 'NOT_Top2015PermTypNum', 'NOT_Top2015PermTypdollars_FOR_addr_avg', 'NOT_Top2015PermTypNum_FOR_hmil_avg', 'NOT_Top2015PermTypNum_FOR_addr_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_avg', 'NOT_Top2015PermTypdollars_FOR_ctrt_avg', 'NOT_Top2015PermTypdollars_FOR_nbhd_avg', 'NOT_Top2015PermTypNum_FOR_ctrt_avg', 'NOT_Top2015PermTypNum_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_avg', 'NOT_Top2015PermTypdollars_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_cnt', 'NOT_Top2015PermTypNum_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_hmil_cnt', 'NOT_Top2015PermTypdollars_FOR_ctrt_cnt', 'NOT_Top2015PermTypNum_FOR_nbhd_cnt', 'NOT_Top2015PermTypdollars_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_cnt', 'NOT_Top2015PermTypNum_FOR_addr_med', 'NOT_Top2015PermTypdollars_FOR_hmil_med', 'NOT_Top2015PermTypNum_FOR_hmil_med', 'NOT_Top2015PermTypdollars_FOR_nbhd_med', 'NOT_Top2015PermTypNum_FOR_nbhd_med', 'NOT_Top2015PermTypdollars_FOR_ctrt_med', 'NOT_Top2015PermTypNum_FOR_ctrt_med', 'Top2016PermTypNum', 'Top2016PermTypdollars', 'Thr12016_Recycling-Pickup', 'Thr12016_excluded', 'Thr12016_Building', 'Thr12016_Street-Sidewalk', 'Thr12016_Trees', 'Thr12016_Enviro', 'Thr12016_Graffiti', 'Thr12016_Housing', 'NOT_Top2016PermTypdollars', 'NOT_Top2016PermTypNum']
model_file.drop(YR_2016_DROPS, inplace = True, axis = 1)

In [47]:
model_file.shape

(89334, 1534)

In [48]:
len(model_file['FIPS'].value_counts())

181

In [49]:
model_file['neighborhood'].value_counts()

Dorchester                                      16246
West Roxbury                                     9412
Hyde Park                                        8284
Roxbury                                          7836
Allston / Brighton                               6892
Greater Mattapan                                 6443
East Boston                                      6152
Jamaica Plain                                    5848
Roslindale                                       5833
South Boston / South Boston Waterfront           5114
Charlestown                                      2316
South End                                        1658
Back Bay                                         1418
Downtown / Financial District                     933
Beacon Hill                                       572
Fenway / Kenmore / Audubon Circle / Longwood      535
Mission Hill                                      401
Boston                                            161
South Boston                

In [50]:
print model_file.shape
low_neighborhoods = ['Allston','Beacon Hill','Boston','Brighton','Chestnut Hill','Downtown / Financial District','Fenway / Kenmore / Audubon Circle / Longwood','Mattapan','Mission Hill','South Boston']
model_file = model_file[~model_file['neighborhood'].isin(low_neighborhoods)]
print model_file.shape

(89334, 1534)
(86598, 1534)


In [51]:
##
# Save out 'Year Solar, PID and is_solar' to use for later analysis 
#model_file[['PID','Year_SOLAR','is_solar']].to_csv('/Users/DavidStier/Downloads/PID_YR_SOLAR_all.csv')
mf2016 = model_file[model_file['Year_SOLAR'] == 2016]
mf2016.to_csv('/Users/DavidStier/Downloads/YR_SOLARis2016QMile7.csv')

In [52]:
tf = model_file  # tf for test_file

In [53]:
tf.fillna(0, inplace = True)

### From here on, only SINGLE FAMILY OWNER OCCUPIED residential are included 

#### See prior notebooks for how to divide model_file for other home sizes


In [54]:
tf_r = tf[tf['resid_bldg']==True]
tf_r_single = tf_r[tf_r['LU'] == 'Residential Single Family']
print tf_r_single.shape, 'single family only'


(30033, 1534) single family only


In [55]:
tf_r_single = tf_r_single[tf_r_single['OWN_OCC'] == 1]
print tf_r_single.shape, 'single family only, now Owner Occ only'
# tfr_dtypes = tf_r.dtypes.tolist()
# tfrcolslist = tf_r.columns.tolist()


(25303, 1534) single family only, now Owner Occ only


## save out copy with geo information


In [56]:
Geotest_file = tf_r_single

In [57]:
Geotest_file.shape

(25303, 1534)

In [58]:
tf_r_single.shape

(25303, 1534)

In [59]:
pause to do geo testing

SyntaxError: invalid syntax (<ipython-input-59-c705537ffa1a>, line 1)

<a id='the_destination'></a>

## The destination

## Segment into train / test / validate files

In [60]:
## 
#  Remove last remaining columns that should be excluded from the dummification
##

final_cols_to_drop = ['Unnamed: 0',
 'GIS_ID',
 'Qmile_grid_cell',
 'hmile_grid_cell',                     
 'PID',
 'PID_LONG',
 'PTYPE',
 'geometry',
 'centroids',
 'st_name_w_type',
 'addr_group',
 'Parcel_ID',
 'Property_Type',
 'parcel_num',
 'FIPS',
 'Latitude',
 'Longitude',
 'YR_BUILT_VS_addr_gr_MED',  # these went to infinity, so will drop
'YR_BUILT_VS_Qmil_gr_MED',
'YR_BUILT_VS_ctrt_gr_MED',
'GROSS_AREA_VS_addr_gr_MED',
'GROSS_AREA_VS_Qmil_gr_MED',
'GROSS_AREA_VS_ctrt_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_addr_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_nbhd_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_Qmil_gr_MED',
'Thr12011to2013_Recycling-Pickup_VS_ctrt_gr_MED'
] 

tf_r_single.drop(final_cols_to_drop, inplace = True, axis = 1)


In [61]:
archive_cols = ['GIS_ID',
 'Qmile_grid_cell',
 'hmile_grid_cell',
 'PID',
 'PID_LONG',
 'PTYPE',
 'geometry',
 'centroids',
 'st_name_w_type',
 'addr_group',
 'Parcel_ID',
 'Property_Type',
 'parcel_num',
 'FIPS',
 'Latitude',
 'Longitude' ]
##
# output a csv of the dropped columns for later audits
##
model_file[['GIS_ID',
 'Qmile_grid_cell',
 'hmile_grid_cell',
 'PID',
 'PID_LONG',
 'PTYPE',
 'geometry',
 'centroids',
 'st_name_w_type',
 'addr_group',
 'Parcel_ID',
 'Property_Type',
 'parcel_num',
 'FIPS',
 'Latitude',
 'Longitude']].to_csv('/Users/DavidStier/Downloads/FinalColsDropped7.csv')

In [62]:
print tf_r_single.shape, 'single family BEFORE dummied'


(25303, 1507) single family BEFORE dummied


In [63]:
## 
# Create file specific column suffixes for the get_dummies functions
##

categ_cols_single = tf_r_single.select_dtypes(include=['object']).columns
print categ_cols_single

Index([u'LU', u'R_AC', u'R_BLDG_STY', u'R_EXT_FIN', u'R_HRAT_TYP',
       u'R_ROOF_TYP', u'STRUCTURE_', u'Bath_Style_1', u'Bath_Style_2',
       u'Bath_Style_3', u'Exterior_Condition', u'Exterior_Wall', u'Grade',
       u'Heat_Type', u'Interior_Condition', u'Interior_Finish',
       u'Kitchen_Style', u'Overall_Condition', u'View', u'Topography',
       u'Outbldg_1', u'Outbldg_2', u'Outbldg_3', u'neighborhood'],
      dtype='object')


In [64]:
tf_r_single_dummied = pd.get_dummies(tf_r_single, columns = categ_cols_single)

print tf_r_single_dummied.shape, 'single family dummied'


(25303, 1707) single family dummied


## Run regression tests, holding out 2016 conversions

In [65]:

validate_file_single = tf_r_single_dummied[tf_r_single_dummied['Year_SOLAR']==2016]
validate_file_single.drop(['is_solar','Year_SOLAR'], axis = 1, inplace = True)


/Applications/anaconda/envs/dsi/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [66]:
train_test_combo_single = tf_r_single_dummied[tf_r_single_dummied['Year_SOLAR']!=2016]
print 'SINGLE FAMILY:  '
print validate_file_single.shape, ' validate file'
print train_test_combo_single.shape, ' all others'

SINGLE FAMILY:  
(219, 1705)  validate file
(25084, 1707)  all others


In [67]:
validate_file_single.to_csv('/Users/DavidStier/Downloads/QMileValidate_full_file7.csv')
train_test_combo_single.to_csv('/Users/DavidStier/Downloads/TrainTestComboFullFile_single_own_occ8.csv')

In [68]:
colnames = train_test_combo_single.columns.tolist()
pd.DataFrame(colnames).to_csv('/Users/DavidStier/Downloads/TrainTestComboFullFile_single_own_occ8COLS.csv')

In [69]:
Y_single = train_test_combo_single['is_solar']


### GET X

In [70]:
tt_single_copy = train_test_combo_single.copy()
tt_single_copy.drop(['is_solar','Year_SOLAR'], axis = 1, inplace = True)
X_single = tt_single_copy

print X_single.shape, ': X_single shape'

(25084, 1705) : X_single shape


## Create 2016 train/test dataset

### Stratified Split Search

In [71]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)
# imp.fit(X)  - this was the original
# X2 = imp.transform(X)

imp.fit(X_single)
X2_single = imp.transform(X_single)

# Note - set X2 as variable for imputed "X" so that I could keep "X" clean

## Single Family Residential - Logit & RFC

In [72]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.5, random_state=0)
sss.get_n_splits(X_single, Y_single)

for train_index, test_index in sss.split(X_single, Y_single):
    xtrain_single, xtest_single = X_single.iloc[train_index], X_single.iloc[test_index]
    ytrain_single, ytest_single = Y_single.iloc[train_index], Y_single.iloc[test_index]
#    logit_single = LogisticRegression()
#    logit_single.fit(xtrain_single, ytrain_single)

print xtrain_single.shape, ytrain_single.shape, xtest_single.shape, ytest_single.shape



(12542, 1705) (12542,) (12542, 1705) (12542,)


In [73]:
xtrain_single_maxValues = xtrain_single.max().tolist()
pd.DataFrame(xtrain_single_maxValues).to_csv('/Users/DavidStier/Downloads/xtrain_single_maxValues.csv')


In [74]:
xtrain_single_cols = xtrain_single.columns.tolist()
pd.DataFrame(xtrain_single_cols).to_csv('/Users/DavidStier/Downloads/xtrain_single_cols.csv')

In [75]:
##
# send copy of feature labels to csv:
#
xtrain_single_column_labels = pd.DataFrame(xtrain_single[0:1])
#xtrain_single_column_labels
xtrain_single_column_labels.to_csv('/Users/DavidStier/Downloads/Qmilextrain_single_column_labels_7.csv')
xtrain_row_indexes_wAV_BLDG = pd.DataFrame(xtrain_single['AV_BLDG'])
xtrain_row_indexes_wAV_BLDG.to_csv('/Users/DavidStier/Downloads/Qmilextrain_row_indexes_wAV_BLDG7.csv')

In [76]:
logit_single.fit(X2_single, Y_single)
logit_single.get_params
logit_ypred_single = logit_single.predict(xtest_single)
cls_report_logit_single = classification_report(ytest_single, logit_ypred_single)
print cls_report_logit_single


NameError: name 'logit_single' is not defined

In [ ]:
confusion_matrix(ytest_single,logit_ypred_single)


### Random Forest Classifier - Single family

In [77]:
rfc_single = RandomForestClassifier(bootstrap= True, 
                                    n_estimators=1000, 
                                    min_samples_split= 4, 
                                    criterion= 'gini', 
                                    max_features=.3, 
                                    random_state = 42, 
                                    max_depth = 5 )

#scoresrfc = cross_val_score(rfc, X, Y, cv=10)
#scoresrfc(X)
rfc_single.fit(xtrain_single, ytrain_single)
rfc_single.get_params
rfc_ypred_single = rfc_single.predict(xtest_single)
cls_report_RFC_single = classification_report(ytest_single, rfc_ypred_single)
print cls_report_RFC_single

print confusion_matrix(ytest_single,rfc_ypred_single)



             precision    recall  f1-score   support

          0       0.96      1.00      0.98     12039
          1       0.60      0.01      0.01       503

avg / total       0.95      0.96      0.94     12542

[[12037     2]
 [  500     3]]


In [78]:
rfc_ysingle_score = rfc_single.predict_proba(xtest_single)
roc_auc_score(ytest_single, rfc_ysingle_score[:, 1])

0.88993004346212778

In [79]:
rfc_ysingle_score[:, 1]

array([ 0.02522884,  0.07584102,  0.01048908, ...,  0.02899261,
        0.02154691,  0.00531551])

In [80]:
rfc_single_features_df = pd.DataFrame(rfc_single.feature_importances_)
rfc_single_features_df.to_csv('/Users/DavidStier/Downloads/QmileRFC_single_featuresWednesday8.csv')


In [81]:
rfc_single_predict_probabs = pd.DataFrame(rfc_ysingle_score)
rfc_single_predict_probabs.to_csv('/Users/DavidStier/Downloads/Qmilerfc_single_predict_probabs Wednesday8.csv')


In [82]:
rfc_single_validate_predict_probs = pd.DataFrame(rfc_single.predict_proba(validate_file_single))
rfc_single_validate_predict_probs.to_csv('/Users/DavidStier/Downloads/Qmilerfc_single_VALIDATE_predict_probabs_WEDS8.csv')


In [83]:
xtest_single.to_csv('/Users/DavidStier/Downloads/Qmilextest_dfWEDNESDAY8.csv')
xtrain_single.to_csv('/Users/DavidStier/Downloads/Qmilextrain_dfWEDNESDAY8.csv')
ytrain_single.to_csv('/Users/DavidStier/Downloads/Qmileytrain_dfWEDNESDAY8.csv')
ytest_single.to_csv('/Users/DavidStier/Downloads/Qmileytest_dfWEDNESDAY8.csv')

## Predict versus actual


In [ ]:
pause here

### GridSearch on Random Forest


In [ ]:
rfc_gs = RandomForestClassifier()

X, y = xtrain_single, ytrain_single
model = RandomForestClassifier(random_state=30)
param_grid = {"n_estimators"      : [100, 500, 1000, 5000],
              "criterion"         : ["gini", "entropy"],
              "max_features": ['sqrt'],
              "max_depth"         : [5, 10, 20],
              "min_samples_split" : [2, 4] ,
              "bootstrap": [True]}
grid_search = GridSearchCV(rfc_gs, param_grid, n_jobs=-1, cv=2)
grid_search.fit(X, y)

print grid_search.best_params_
#print confusion_matrix(ytest_SFR_meth1,rfc_ypred_SFR_meth1)

rfc_gs.get_params
rfc_gs_ypred_single = rfc_gs_single.predict(xtest_single)
cls_report_RFC_gs_single = classification_report(ytest_single, rfc_gs_ypred_single)
print cls_report_RFC_gs_single

print confusion_matrix(ytest_single,rfc_gs_ypred_single)

rfc_single_features_df = pd.DataFrame(rfc_gs.feature_importances_)
rfc_single_features_df.to_csv('/Users/DavidStier/Downloads/QmileRFC_gs_single_features7.csv')




In [ ]:
# xsingleMaxes = pd.DataFrame(X_single.max())
# xsingleMaxes.to_csv('/Users/DavidStier/Downloads/X_singlemaxex.csv')
# X_single[0:4].to_csv('/Users/DavidStier/Downloads/X_singleColNames.csv')

In [ ]:
# rfc_ysingle_score_df = pd.DataFrame(rfc_ysingle_score)
# rfc_ysingle_score_df.to_csv('/Users/DavidStier/Downloads/rfc_ysingle_score_dfTues.csv')

In [ ]:
# tf_r_single_dummied.shape

In [ ]:
#LU_solar_group = model_file.groupby(['LU','is_solar','OWN_OCC'])

In [ ]:
#LU_solar_group['Year_SOLAR'].value_counts()

In [ ]:
tf_r_single_dummied.YR_BUILT.value_counts()

In [ ]:
from shapely.ops import cascaded_union

In [ ]:
Geotest_file['geometry']

In [ ]:
# Geotest_file = Geotest_file.rename(columns = {'geometry' : 'parcel_shape'})
# geo_df = geo_df.rename(columns={'geometry': 'point_Lat_Lon'}).set_geometry('point_Lat_Lon')
# geo_df['geometry']

In [ ]:
from geopandas import GeoDataFrame

crs = "epsg:4326"
geo_df = GeoDataFrame(Geotest_file, crs=crs)

In [ ]:
geo_df.geometry

In [ ]:
Geotest_file['addr_group'][0:11]

In [ ]:
Geotest_file['addr_group_shape'] = cascaded_union(Sorted_add['geometry'])

In [ ]:
Sorted_add = Geotest_file.sort(['addr_group'], ascending=[1])

In [ ]:
type(Sorted_add)

In [ ]:
cur_addr_group = ''
addr_group_shape = []
for i in Sorted_add['addr_group'][0:100]:
    print i
    if i = cur_addr_group:
        addr_group_shape = 
    

In [ ]:
f = plt.figure(figsize=(15, 8))
ax = f.gca()
geo_df.geometry[0:11].plot(ax=ax)
mpl.display(fig=f)

In [ ]:
type(geo_df)

In [ ]:
geo_df.shape

In [ ]:
type(geo_df['geometry'][11:12])

In [ ]:
geo_df.columns[31:63]